### Day 1

In [ ]:
# Cleans Input into list1, list2 - each an int list

file = open("day1.input", "r")
input = file.read()

file.close()

nums = [*map(lambda x: int(x), input.split())]
list1, list2 = nums[0::2], nums[1::2]
list1[:10], list2[:10]

In [ ]:
# Part 1

ans = sum(map(lambda x, y: abs(x - y) , sorted(list1), sorted(list2)))	# Sorts each list and sums the absolute differences between elements
ans																	  	

In [ ]:
# Part 2

hash = {}

for num in list2:
	hash[num] = hash.get(num, 0) + 1					# For each number in list 2, counts each number

ans = sum(map(lambda x: x * hash.get(x, 0), list1))		# Iterate through list 1 and sum the products between the elements and counts
ans

### Day 2

In [ ]:
# Cleans Input into nums, an array of int lists

file = open("day2.input", "r")

nums = [[*map(lambda x: int(x), line.split())] for line in file]

file.close()

nums[:10]

In [ ]:
# Part 1

def safe(lst):                                                      # Safe function dtermines if a given list is safe
    diffs = [lst[i + 1] - lst[i] for i in range(len(lst) - 1)]      # Creates an intermediate difference list between consecutive elements
    return all(map(lambda x: 0 < x < 4, diffs))                     # A list is safe iff all differences are between 1 and 3 inclusive

ans = sum([safe(line) or safe(line[::-1]) for line in nums])        # Sums the number of safe lists (uses both the list or the reverse to allow for strictly inc and dec)
ans

In [ ]:
# Part 2

def safe_h(lst):                                                                # An ammended safety checker that allows for one mistake
    for i in range(len(lst) - 1):                                               
        if not 0 < lst[i + 1] - lst[i] < 4:                                     # On the first mistake,
            return safe(lst[:i] + lst[i+1:]) or safe(lst[:i+1] + lst[i+2:])     # Check either of the offending elements and returns if removing either one makes the list safe
    return True

ans = sum([safe_h(line) or safe_h(line[::-1]) for line in nums])                # Once again, return the number of lists in which removing 0 or 1 element are safe.
ans

### Day 3

In [ ]:
# Gets input as a string

file = open("day3.input", "r")
input = file.read()

file.close()
input[:100]

In [ ]:
# Part 1

import re

matches = re.finditer(r"mul\((\d{1,3}),(\d{1,3})\)", input)			# Finds all of the mul() clauses and captures the digits
ans = sum([int(m.group(1)) * int(m.group(2)) for m in matches])		# Sums the products of the digits for each match
ans

In [ ]:
# Part 2

ans, do = 0, True
matches = re.finditer(r"mul\((\d{1,3}),(\d{1,3})\)|do\(\)|don't\(\)", input)	# Finds all the mul(), do(), and don't() clauses
for m in matches:
	match m.group(0):															# Matches the clause with...
		case 'do()':															# DO: set flag to true
			do = True
		case "don't()":															# DON'T: set flag to false
			do = False
		case _:																	# default (mult): if the flag is true, add the product to accumulator
			ans += int(m.group(1)) * int(m.group(2)) if do else 0

ans

### Day 4

In [ ]:
# Gets input as an n length array of length n strings

file = open("day4.input", "r")
input = file.read().split()

file.close()
input[:2]

In [ ]:
# Part 1

ans = 0
d = [(x, y) for x in (-1, 0, 1) for y in (-1, 0, 1)]                                                                # list of all directional pairs (1, 1), (0, -1) etc. in 2D
find = "XMAS"

def safe_index(M, default, i, j):                                                                                   # function that returns default if out of bounds; else, indexes properly
    x, y = len(M) - 1, len(M[0]) - 1
    return M[i][j] if (0 <= i <= x) and (0 <= j <= y) else default

for i in range(len(input)):                                                                                         
    for j in range(len(input[0])):
        if input[i][j] == "X":                                                                                      # For each coordinate beginning with X...
            for (x, y) in d:                                                                                        # Look for "XMAS" in each direction
                ans += "".join([safe_index(input, "", i + x * k, j + y * k) for k in range(len(find))]) == find     # If XMAS is found, add one to the accumulator

ans

In [ ]:
# Part 2

ans = 0

for i in range(1, len(input) - 1):
    for j in range(1, len(input[0]) - 1):
        if input[i][j] == "A":                                                      # For each coordinate with A...
            ans += (                                                                # Add one to accumulator if M and S are in opposite diagonals for both diagonals
                (input[i + 1][j + 1] == "M" and input[i - 1][j - 1] == "S")
                or (input[i + 1][j + 1] == "S" and input[i - 1][j - 1] == "M")
            ) and (
                (input[i + 1][j - 1] == "M" and input[i - 1][j + 1] == "S")
                or (input[i + 1][j - 1] == "S" and input[i - 1][j + 1] == "M")
            )

ans

### Day 5

In [ ]:
file = open("day5rules.input", "r")
rules = [[*map(int, rule.split("|"))] for rule in file.read().split()]

file.close()

file = open("day5queries.input", "r")
queries = [[*map(int, query.split(","))] for query in file.read().split()]

file.close()

rules[:5], queries[:5]

In [ ]:
# Part 1

from collections import defaultdict

rule_map = defaultdict(set)							# Creates an adjacency dict of rules
for n1, n2 in rules:
	rule_map[n1].add(n2)

def process_query(query):							# Makes sure there are no backwards edges in the ordering
	seen = set()									
	for num in query:								# For every number in query,
		if (seen & rule_map[num]):					# If there is an edge pointing behind (invalid order), return 0
			return 0
		seen.add(num)
	return query[len(query) // 2]					# If all vertices are good, return the middle element

sum([process_query(query) for query in queries])	# Sums the middle elements of the valid queries

In [ ]:
# Part 2

from graphlib import TopologicalSorter

def process_query(query):
    seen = set()
    all = set(query)
    for num in query:
        if seen & rule_map[num]:                                                                # If there is an invalid
            topsort = TopologicalSorter(dict([[n, list(rule_map[n] & all)] for n in query]))    # Sorts the elements
            sorted = [*topsort.static_order()]

            return sorted[len(sorted) // 2]                                                     # Return the middle element of the now-valid ordering
        seen.add(num)
    return 0                                                                                    # Valid orderings return 0

sum([process_query(query) for query in queries])                                                # Sums the middle elements of fixed orderings

### Day 6

In [ ]:
file = open("day6.input", "r")
input = file.read().split()

file.close()
input[:10]

In [ ]:
# Part 1

grid = {(x, y): (-1 if v == "#" else 0) for (x, r) in enumerate(input) for (y, v) in enumerate(r)}
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]
r, x, y = 0, 0, 0
ans = 0

for i, row in enumerate(input):
	if "^" in row:
		x, y = i, row.find("^")
		grid[x, y] = 0
		break

while (x, y) in grid:
    if not grid[x, y]:
        grid[x, y] = 1
        ans += 1

    nextx, nexty = x + dirs[r][0], y + dirs[r][1]
    while grid.get((nextx, nexty), 1) == -1:
        r = (r + 1) % 4
        nextx, nexty = x + dirs[r][0], y + dirs[r][1]
    x, y = nextx, nexty

ans

In [ ]:
# Part 2

grid = {(x, y): (-1 if v == "#" else 0) for (x, r) in enumerate(input) for (y, v) in enumerate(r)}
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]
p = [2, 3, 5, 7]
r, x, y = 0, 0, 0
ans = 0

for i, row in enumerate(input):
    if "^" in row:
        x, y = i, row.find("^")
        grid[x, y] = 0
        break

# TODO: Clean up this monstrosity, holy hell
while (x, y) in grid:
    if not grid[x, y]:
        grid[x, y] = 1

    nextx, nexty = x + dirs[r][0], y + dirs[r][1]

    while (nextx, nexty) in grid and grid[nextx, nexty] == -1:
        r = (r + 1) % 4
        nextx, nexty = x + dirs[r][0], y + dirs[r][1]

    if (nextx, nexty) in grid and not grid[nextx, nexty]:
        new_grid = dict(grid)
        new_grid[nextx, nexty] = -1
        x_e, y_e, r_e = x, y, r
        while (x_e, y_e) in new_grid:
            if not new_grid[x_e, y_e]:
                new_grid[x_e, y_e] = p[r_e]
            elif not (new_grid[x_e, y_e] % p[r_e]):
                ans += 1
                break

            nextx_e, nexty_e = x_e + dirs[r_e][0], y_e + dirs[r_e][1]

            while (nextx_e, nexty_e) in new_grid and new_grid[nextx_e, nexty_e] == -1:
                r_e = (r_e + 1) % 4
                nextx_e, nexty_e = x_e + dirs[r_e][0], y_e + dirs[r_e][1]

            new_grid[x_e, y_e] *= p[r_e]
            x_e, y_e = nextx_e, nexty_e

    grid[x, y] *= p[r]
    x, y = nextx, nexty

ans

### Day 7

In [ ]:
file = open("day7.input", "r")
input = [(int(k), [*map(int, v.split())]) for (k, v) in [row.split(": ") for row in file.read().split("\n")]]

file.close()
input[:10]

In [ ]:
# Part 1

def makeable(target, nums):
    if len(nums) < 2 or nums[0] > target:
        return nums[0] == target

    s, p = nums[0] + nums[1], nums[0] * nums[1]
    return makeable(target, [s, *nums[2:]]) or makeable(target, [p, *nums[2:]])

ans = sum([t if makeable(t, l) else 0 for t, l in input])
ans

In [ ]:
# Part 2

def makeable(target, nums):
    if len(nums) < 2 or nums[0] > target:
        return nums[0] == target

    s, p, c = nums[0] + nums[1], nums[0] * nums[1], int(str(nums[0]) + str(nums[1]))
    return (
        makeable(target, [s, *nums[2:]])
        or makeable(target, [p, *nums[2:]])
        or makeable(target, [c, *nums[2:]])
    )

ans = sum([t if makeable(t, l) else 0 for t, l in input])
ans

### Day 8

In [ ]:
# Gets input as a list of row strings

file = open("day8.input", "r")
input = file.read().split()

file.close()
input[:10]

In [ ]:
# Part 1

from collections import defaultdict

nodes = set()
towers, grid = defaultdict(list), dict()


for x, row in enumerate(input):
    for y, s in enumerate(row):
        grid[x, y] = 1 if s == "." else 0
        if s != ".":
            towers[s].append((x, y))

for l in towers.values():
    for i in l:
        for j in l:
            if i != j:
                x1, y1 = i
                x2, y2 = j
                dx, dy = x2 - x1, y2 - y1

                pos1, pos2 = (x1 - dx, y1 - dy), (x2 + dx, y2 + dy)
                if pos1 in grid:
                    nodes.add(pos1)
                if pos2 in grid:
                    nodes.add(pos2)
                
ans = len(nodes)
ans

In [ ]:
nodes = set()
towers, grid = defaultdict(list), dict()

for x, row in enumerate(input):
    for y, s in enumerate(row):
        grid[x, y] = 1 if s == "." else 0
        if s != ".":
            towers[s].append((x, y))
            nodes.add((x, y))

for l in towers.values():
    for i in l:
        for j in l:
            if i != j:
                x1, y1 = i
                x2, y2 = j
                dx, dy = x2 - x1, y2 - y1

                pos1, pos2 = (x1 - dx, y1 - dy), (x2 + dx, y2 + dy)
                while pos1 in grid:
                    nodes.add(pos1)
                    pos1 = (pos1[0] - dx, pos1[1] - dy)
                while pos2 in grid:
                    nodes.add(pos2)
                    pos2 = (pos2[0] + dx, pos2[1] + dy)

ans = len(nodes)
ans

### Day 9

In [ ]:
file = open("day9.input", "r")
input = [([i // 2 if i % 2 == 0 else -1] * int(c)) for i, c in enumerate(file.read())]

file.close()

input[:10]

In [ ]:
bits = [bit for space in input for bit in space]
i, j = 0, len(bits) - 1
ans = 0

while bits[j] == -1:
	j -= 1

while i <= j:
	if bits[i] == -1:
		ans += i * bits[j]
		j -= 1
		while bits[j] == -1:
			j -= 1
	else:
		ans += i * bits[i]
	i += 1

ans

In [ ]:
from collections import defaultdict
import heapq

bits = [bit for space in input for bit in space]
files, free = [], defaultdict(list)
tot = 0
out = len(bits) + 1

# TODO: Clean this up. @_@
for i, space in enumerate(input):
	size = len(space)
	if i % 2 == 0:
		files.append((tot, size, i // 2))
	else:
		free[size].append((tot, size))
	tot += size

mins = [heapq.heappop(free[k]) if free[k] else (out, -1) for k in range(10)]

def move(pos, b = bits, m = mins, f = free):
    start, size, num = pos
    move_to, free_size = min(m[size:])
    if free_size > 0 and move_to < start:
        b[start : start + size] = [0] * size
        b[move_to : move_to + size] = [num] * size
        m[free_size] = heapq.heappop(f[free_size]) if f[free_size] else (out, -1)

        remaining = free_size - size
        if remaining:
            new_start = move_to + size
            new_space = (new_start, remaining)

            if new_start < m[remaining][0]:
                heapq.heappush(f[remaining], m[remaining])
                m[remaining] = new_space
            else:
                heapq.heappush(f[remaining], new_space)

while files:
    move(files.pop())

ans = sum([i * b if b > 0 else 0 for i, b in enumerate(bits)])
ans

### Day 10

In [ ]:
file = open("day10.input", "r")
input = { (i, j): int(num) for i, r in enumerate(file.read().split()) for j, num in enumerate(r) }

file.close()

[*input.items()][:10]

In [ ]:
ans = 0

def dfs(start, l, peaks):
	num = input.get(start, -1)
	if num != l:
		return
	if num == 9:
		peaks.add(start)
		return
	i, j = start
	dfs((i + 1, j), l + 1, peaks)
	dfs((i - 1, j), l + 1, peaks)
	dfs((i, j + 1), l + 1, peaks)
	dfs((i, j - 1), l + 1, peaks)
	return

for k, v in input.items():
	if not v:
		peaks = set()
		dfs(k, 0, peaks)
		ans += len(peaks)
ans

In [ ]:
ans = 0

def dfs(start, l):
    num = input.get(start, -1)
    if num != l:
        return 0
    if num == 9:
        return 1
    i, j = start
    return (
        dfs((i + 1, j), l + 1)
        + dfs((i - 1, j), l + 1)
        + dfs((i, j + 1), l + 1)
        + dfs((i, j - 1), l + 1)
    )


for k, v in input.items():
    if not v:
        ans += dfs(k, 0)
ans

### Day 11

In [ ]:
# Gets input as a list of strings

file = open("day11.input", "r")
input = [*map(int, file.read().split())]

file.close
input

In [ ]:
def calc_stones(num, level):
    if level < 25:
        digits = len(str(num))

        if digits % 2:
            if not num:
                return calc_stones(1, level + 1)
            return calc_stones(num * 2024, level + 1)
        else:
            half = digits // 2
            return calc_stones(num // (10**half), level + 1) + calc_stones(
                num % (10**half), level + 1
            )
    return 1

ans = sum([calc_stones(num, 0) for num in input])
ans

In [ ]:
memo = {}

def memoize(num, left, val):
    memo[(num, left)] = val
    return val

def calc_stones(num, level):
    left = 75 - level
    if (num, left) in memo:
        return memo[(num, left)]

    if level < 75:
        digits = len(str(num))

        if digits % 2:
            if not num:
                return memoize(num, left, calc_stones(1, level + 1))
            return memoize(num, left, calc_stones(num * 2024, level + 1))
        else:
            half = digits // 2
            return memoize(num, left, calc_stones(num // (10**half), level + 1) + calc_stones(num % (10**half), level + 1))
    return 1


ans = sum([calc_stones(num, 0) for num in input])
ans

### Day 12

In [3]:
file = open("day12.input", "r")
input = { (i, j): t for i, r in enumerate(file.read().split()) for j, t in enumerate(r) }

file.close()

[*input.items()][:10]

[((0, 0), 'Y'),
 ((0, 1), 'Y'),
 ((0, 2), 'Y'),
 ((0, 3), 'Y'),
 ((0, 4), 'Y'),
 ((0, 5), 'M'),
 ((0, 6), 'M'),
 ((0, 7), 'M'),
 ((0, 8), 'M'),
 ((0, 9), 'M')]

In [16]:
grid = dict(input)
dims = []
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]

def dfs(pos, v):
    i, j = pos
    curr = grid.get(pos, "")
    if curr == curr.lower() or curr != v:
        return (0, 0)

    grid[pos] = curr.lower()
    p = 4 - sum(2 * (grid.get((i + di, j + dj), "") == v.lower()) for di, dj in dirs)
    a_rest, p_rest = 0, 0
    for di, dj in dirs:
        a_part, p_part = dfs((i + di, j + dj), v)
        a_rest += a_part
        p_rest += p_part

    return (1 + a_rest, p + p_rest)

dims = [dfs(k, v) for k, v in grid.items()]

ans = sum(a * p for a, p in dims)
ans

[(48, 42), (0, 0), (0, 0), (0, 0), (0, 0), (138, 84), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (80, 64), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (4, 10), (0, 0), (0, 0), (0, 0), (0, 0), (8, 16), (0, 0), (0, 0), (0, 0), (0, 0), (131, 104), (0, 0), (1, 4), (0, 0), (12, 18), (0, 0), (0, 0), (0, 0), (0, 0), (72, 64), (0, 0), (2, 6), (0, 0), (0, 0), (0, 0), (29, 30), (0, 0), (0, 0), (0, 0), (0, 0), (93, 104), (24, 32), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (145, 74), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (54, 52), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (58, 52), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (113, 72), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (92, 62), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0,

In [21]:
grid = dict(input)
dims = []
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]
diags = [(i, j) for i in (-1, 1) for j in (-1, 1)]

def is_corner(i, j, d, v):
    di, dj = d
    adj1, adj2, diag = (
        grid.get((i + di, j), "").lower(),
        grid.get((i, j + dj), "").lower(),
        grid.get((i + di, j + dj), "").lower(),
    )
    return (adj1 != v and adj2 != v) or (adj1 == adj2 == v and diag != v)


def dfs(pos, v):
    i, j = pos
    curr = grid.get(pos, "")
    if curr == curr.lower() or curr != v:
        return (0, 0)

    grid[pos] = curr.lower()
    p = sum(is_corner(i, j, d, curr.lower()) for d in diags)
    a_rest, p_rest = 0, 0
    for di, dj in dirs:
        a_part, p_part = dfs((i + di, j + dj), v)
        a_rest += a_part
        p_rest += p_part

    return (1 + a_rest, p + p_rest)


dims = [dfs(k, v) for k, v in grid.items()]

ans = sum(a * p for a, p in dims)
ans

921636